In [9]:
import os, shutil, csv, math
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import butter, lfilter


In [10]:
# This function creates a directory if the directory with that name does not exist, 
# or empties/deletes the contents of the directory if the directory already exists


def replace_65th_column_if_avg_zero(input_path):
    # Open the input CSV file
    with open(input_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)

    # Calculate the average of the 65th column
    sum_65th_column = 0
    num_values = 0

    for row in data:
        try:
            value = float(row[64])  # 65th column index is 64 (0-indexed)
            sum_65th_column += value
            num_values += 1
        except ValueError:
            pass  # Skip non-numeric values

    if num_values == 0:
        average_65th_column = 0
    else:
        average_65th_column = sum_65th_column / num_values

    # Only proceed if the average of the 65th column is zero
    if average_65th_column == 0:
        # Process the data and store modified data in a new list
        modified_data = []

        for row in data:
            try:
                value_64th_column = float(row[63])  # 64th column index is 63 (0-indexed)
                value_57th_column = float(row[56])  # 57th column index is 56 (0-indexed)
                new_value = (value_64th_column + value_57th_column) / 2
                row[64] = str(new_value)
            except ValueError:
                pass  # Skip non-numeric values
            
            modified_data.append(row)

        # Write the modified data to the input file
        with open(input_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(modified_data)
            
            
            
def create_or_empty_directory(directory_path):

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    else:
        
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)


                
# Given a pandas dataframe, the below function looks at each of the columns for a sequence of zeros
# If the number of zeros in the sequence passes a certain threshold, the function considers that channel dead
# All such column indexes are maintained in a list and the list is returned back 

def find_columns_with_consecutive_zeroes(df, consecutive_zeroes_threshold):
    
    zero_columns = []
    for col in df.columns:
        consecutive_zeroes = 0
        max_consecutive_zeroes = 0
        for value in df[col]:
            if value == 0:
                consecutive_zeroes += 1
                max_consecutive_zeroes = max(max_consecutive_zeroes, consecutive_zeroes)
            else:
                consecutive_zeroes = 0
            
            if max_consecutive_zeroes >= consecutive_zeroes_threshold:
                zero_columns.append(col)
                break

    return zero_columns
    
    
    
# A function which tells how many usable channels are present surrounding the channel 
        
def exists_nearby(ch_index, number_of_channels, mult_factor, dead_channels_list):
    
    if number_of_channels*(mult_factor+1) >= ch_index > number_of_channels*mult_factor and ch_index not in dead_channels_list:
        return 1
    else:
        return 0


# This function tells how many usable electrode channels are present nearby the main channel
# It creates a score board associating each of the dead channel indexes with a score, higher the score, 
# higher the number of usable surrounding electrode channels, thus higher the accuracy of the averaging
# We always perform averaging on the electrode channel having the highest score, then calculate the score board again
# Look at the new high score, oerform averaging on that dead channel and go on doing this iteratively

    
def calculate_score_list(dead_channels_list, number_of_channels, grid_height):
    
    score_list = []
    
    for i in dead_channels_list:
    
        mult_factor = math.floor(i/number_of_channels)
        temp = 0

        temp += exists_nearby(i + 1, number_of_channels, mult_factor, dead_channels_list)
        temp += exists_nearby(i - 1, number_of_channels, mult_factor, dead_channels_list)
        temp += exists_nearby(i + grid_height, number_of_channels, mult_factor, dead_channels_list)
        temp += exists_nearby(i - grid_height, number_of_channels, mult_factor, dead_channels_list)
        
        if (i%8 == 0 or i-1%8 == 0):
            temp -= 1
        
        score_list.append(temp)
        
    return score_list
        

    
def rectify_EMG(df, grid_height, grid_width, threshold):
    
    dead_channels_list = find_columns_with_consecutive_zeroes(df, threshold)
    print("List of dead channels : ")
    print(dead_channels_list)
    number_of_channels = grid_height*grid_width
    df_rectified = df
    
    for i in range(len(dead_channels_list)):
        
        print('Remaining Dead Channels : ',dead_channels_list)
        
        score_list = calculate_score_list(dead_channels_list, number_of_channels, grid_height)
        
        print('Dead Channel Score List : ',score_list)
        
        max_index = score_list.index(max(score_list))
        mult_factor = math.floor(dead_channels_list[max_index]/number_of_channels)
        
        if (dead_channels_list[max_index] + grid_height) > df.shape[1] - 2:
            
            sum_values = ((df.iloc[:, dead_channels_list[max_index]+1]*exists_nearby(dead_channels_list[max_index]+1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]-1]*exists_nearby(dead_channels_list[max_index]-1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]-grid_height]*exists_nearby(dead_channels_list[max_index]-grid_height, number_of_channels, mult_factor, dead_channels_list)))

        elif (dead_channels_list[max_index] - grid_height) < 1:
            
            sum_values = ((df.iloc[:, dead_channels_list[max_index]+1]*exists_nearby(dead_channels_list[max_index]+1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]-1]*exists_nearby(dead_channels_list[max_index]-1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]+grid_height]*exists_nearby(dead_channels_list[max_index]+grid_height, number_of_channels, mult_factor, dead_channels_list)))

        else:
            
            sum_values = ((df.iloc[:, dead_channels_list[max_index]+1]*exists_nearby(dead_channels_list[max_index]+1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]-1]*exists_nearby(dead_channels_list[max_index]-1, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]+grid_height]*exists_nearby(dead_channels_list[max_index]+grid_height, number_of_channels, mult_factor, dead_channels_list)) + (df.iloc[:, dead_channels_list[max_index]-grid_height]*exists_nearby(dead_channels_list[max_index]-grid_height, number_of_channels, mult_factor, dead_channels_list)))
      
        if dead_channels_list[max_index] % 8 == 0:
            
            sum_values -= (df.iloc[:, dead_channels_list[max_index]+1])
            
        if dead_channels_list[max_index]-1 % 8 == 0:
            
            sum_values -= (df.iloc[:, dead_channels_list[max_index]-1])
        
        df_rectified.iloc[:, dead_channels_list[max_index]] = sum_values/score_list[max_index]
        dead_channels_list = find_columns_with_consecutive_zeroes(df_rectified,threshold)
    
    return df_rectified


In [11]:

Input_folder = "Processed_Raw_EMG_Data"
Output_folder = "Rectified_EMG_Data"
create_or_empty_directory(Output_folder)

zeros_threshold = 2000
grid_height = 8
grid_width = 4

csv_files = [file for file in os.listdir(Input_folder) if file.endswith(".csv")]


In [12]:

for csv_file in csv_files:
    
    print("\n---> ",csv_file)
    
    input_path = os.path.join(Input_folder, csv_file)
    df = pd.read_csv(input_path, header=None)
    dead_channels_list = find_columns_with_consecutive_zeroes(df, zeros_threshold)
    
    if len(dead_channels_list) > 0:
        
        df_rectified = rectify_EMG(df, grid_height, grid_width, zeros_threshold)
        output_path_rec = os.path.join(Output_folder, csv_file)
        df_rectified.to_csv(output_path_rec, header=False, index=False)
    
    else:
        
        output_path_rec = os.path.join(Output_folder, csv_file)
        df.to_csv(output_path_rec, header=False, index=False)
        
    replace_65th_column_if_avg_zero(output_path_rec)
        



--->  P02_Right_90.csv

--->  P36_Left_Extension3.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P32_Left_Pull3.csv

--->  P30_Left_90.csv
List of dead channels : 
[35]
Remaining Dead Channels :  [35]
Dead Channel Score List :  [3]

--->  P05_Right_90.csv

--->  P06_Left_Flexion.csv

--->  P02_Left_Push2.csv
List of dead channels : 
[42, 49, 50, 51]
Remaining Dead Channels :  [42, 49, 50, 51]
Dead Channel Score List :  [3, 3, 1, 3]
Remaining Dead Channels :  [49, 50, 51]
Dead Channel Score List :  [3, 2, 3]
Remaining Dead Channels :  [50, 51]
Dead Channel Score List :  [3, 3]
Remaining Dead Channels :  [51]
Dead Channel Score List :  [4]

--->  P19_Right_Pull3.csv

--->  P38_Left_120.csv
List of dead channels : 
[7, 8, 23, 24]
Remaining Dead Channels :  [7, 8, 23, 24]
Dead Channel Score List :  [2, 1, 3, 2]
Remaining Dead Channels :  [7, 8, 24]
Dead Channel S

Remaining Dead Channels :  [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 1, 0, 1, 1, 1, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 0, 1, 1, 1, 1, 1, 1, 0, -1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38

Remaining Dead Channels :  [7, 8, 24]
Dead Channel Score List :  [2, 1, 3]
Remaining Dead Channels :  [7, 8]
Dead Channel Score List :  [2, 1]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P38_Right_90.csv
List of dead channels : 
[7, 8, 22, 23, 24]
Remaining Dead Channels :  [7, 8, 22, 23, 24]
Dead Channel Score List :  [2, 1, 3, 2, 2]
Remaining Dead Channels :  [7, 8, 23, 24]
Dead Channel Score List :  [2, 1, 3, 2]
Remaining Dead Channels :  [7, 8, 24]
Dead Channel Score List :  [2, 1, 3]
Remaining Dead Channels :  [7, 8]
Dead Channel Score List :  [2, 1]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P15_Right_Flexion.csv

--->  P25_Left_Extension3.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P36_Left_90.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Re

List of dead channels : 
[8, 19, 24]
Remaining Dead Channels :  [8, 19, 24]
Dead Channel Score List :  [2, 4, 3]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P34_Left_Push2.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P26_Left_Push.csv

--->  P26_Left_60.csv

--->  P06_Left_120.csv

--->  P08_Left_60.csv

--->  P10_Left_Flexion.csv

--->  P14_Right_120.csv

--->  P19_Left_Pull2.csv

--->  P30_Right_Flexion1.csv

--->  P02_Left_Flexion2.csv
List of dead channels : 
[50]
Remaining Dead Channels :  [50]
Dead Channel Score List :  [4]

--->  P08_Right_Push2.csv

--->  P18_Right_Extension.csv

--->  P06_Left_90.csv

--->  P03_Right_120.csv

--->  P35_Right_Push2.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remai

Remaining Dead Channels :  [7, 8, 23, 24]
Dead Channel Score List :  [2, 1, 3, 2]
Remaining Dead Channels :  [7, 8, 24]
Dead Channel Score List :  [2, 1, 3]
Remaining Dead Channels :  [7, 8]
Dead Channel Score List :  [2, 1]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P24_Right_Extension2.csv

--->  P36_Left_60.csv
List of dead channels : 
[8, 24]
Remaining Dead Channels :  [8, 24]
Dead Channel Score List :  [2, 3]
Remaining Dead Channels :  [8]
Dead Channel Score List :  [2]

--->  P16_Left_90.csv
List of dead channels : 
[25]
Remaining Dead Channels :  [25]
Dead Channel Score List :  [3]

--->  P32_Left_Flexion1.csv

--->  P02_Left_Pull3.csv
List of dead channels : 
[42, 49, 50, 51]
Remaining Dead Channels :  [42, 49, 50, 51]
Dead Channel Score List :  [3, 3, 1, 3]
Remaining Dead Channels :  [49, 50, 51]
Dead Channel Score List :  [3, 2, 3]
Remaining Dead Channels :  [50, 51]
Dead Channel Score List :  [3, 3]
Remaining Dead Channels :  [51]
Dead Channel Score

Remaining Dead Channels :  [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [1, 1, 1, 1, 1, 1, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 1, 1, 1, 1, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 1, 1, 1, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Dead Channel Score List :  [2, 1, 1, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining Dead Channels :  [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48

List of dead channels : 
[21]
Remaining Dead Channels :  [21]
Dead Channel Score List :  [4]

--->  P13_Right_Pull2.csv

--->  P16_Right_Extension.csv

--->  P10_Right_60.csv

--->  P03_Left_Extension.csv

--->  P02_Left_60.csv
List of dead channels : 
[23, 33, 35, 36, 38, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 61]
Remaining Dead Channels :  [23, 33, 35, 36, 38, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 61]
Dead Channel Score List :  [4, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2]
Remaining Dead Channels :  [33, 35, 36, 38, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 61]
Dead Channel Score List :  [1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2]
Remaining Dead Channels :  [33, 35, 38, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 61]
Dead Channel Score List :  [1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2]
Remaining Dead Channels :  [33, 38, 41, 42, 43, 45, 46, 49, 50, 51, 52, 53, 61]
Dead Channel Score List :  [1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2]
Remaining Dead Channels :  [33, 41, 42, 43, 45, 46, 